<a href="https://colab.research.google.com/github/tripti196/AlphaPose/blob/master/new_posTagger_MLR_Conllu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!pip install conllu

In [36]:
import conllu

In [82]:
# Load train, dev, and test datasets
def load_conllu(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    return conllu.parse(data)

train_data = load_conllu("/content/drive/MyDrive/NLP drive/UD_Hindi-HDTB/hi_hdtb-ud-train.conllu")
dev_data = load_conllu("/content/drive/MyDrive/NLP drive/UD_Hindi-HDTB/hi_hdtb-ud-dev.conllu")
test_data = load_conllu("/content/drive/MyDrive/NLP drive/UD_Hindi-HDTB/hi_hdtb-ud-test.conllu")
output=open("Desktop",'w', encoding='utf-8')
print(len(train_data),len(dev_data),len(test_data))

13306 1659 1684


In [38]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.metrics import precision_score, recall_score, f1_score

In [39]:
# Function to extraction of words and tags from conllu dataset
def extract_words_and_tags(parsed_data):
    words = [[token['form'] for token in sentence] for sentence in parsed_data]
    tags = [[token['upos'] for token in sentence] for sentence in parsed_data]
    return words, tags

# Extracted the words and tags from diffrent conllu datasets
train_words, train_tags = extract_words_and_tags(train_data)
dev_words, dev_tags = extract_words_and_tags(dev_data)
test_words, test_tags = extract_words_and_tags(test_data)
print(train_words[0], train_tags[0])
print(dev_words[0], dev_tags[0])
print(test_words[0], test_tags[0])

['यह', 'एशिया', 'की', 'सबसे', 'बड़ी', 'मस्जिदों', 'में', 'से', 'एक', 'है', '।'] ['DET', 'PROPN', 'ADP', 'ADV', 'ADJ', 'NOUN', 'ADP', 'ADP', 'NUM', 'AUX', 'PUNCT']
['रामायण', 'काल', 'में', 'भगवान', 'राम', 'के', 'पुत्र', 'कुश', 'की', 'राजधानी', 'कुशावती', 'को', '483', 'ईसा', 'पूर्व', 'बुद्ध', 'ने', 'अपने', 'अंतिम', 'विश्राम', 'के', 'लिए', 'चुना', '।'] ['PROPN', 'PROPN', 'ADP', 'NOUN', 'PROPN', 'ADP', 'NOUN', 'PROPN', 'ADP', 'NOUN', 'PROPN', 'ADP', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'ADP', 'PRON', 'ADJ', 'NOUN', 'ADP', 'ADP', 'VERB', 'PUNCT']
['इसके', 'अतिरिक्त', 'गुग्गुल', 'कुंड', ',', 'भीम', 'गुफा', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'स्थल', 'हैं', '।'] ['PRON', 'ADP', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'PART', 'ADJ', 'NOUN', 'AUX', 'PUNCT']


In [40]:
print(len(train_words), len(train_tags))
print(len(dev_words), len(dev_tags))
print(len(test_words), len(test_tags))

13306 13306
1659 1659
1684 1684


In [41]:
# Build a vocabulary for words and tags
import torchtext
from torchtext.vocab import build_vocab_from_iterator

def build_vocab(data, min_freq=1):
    specials = ['<unk>', '<pad>']  # Add '<unk>' for unknown words to avoid the missing error
    return build_vocab_from_iterator(data, min_freq=min_freq, specials=specials)

word_vocab = build_vocab(train_words)
tag_vocab = build_vocab(train_tags)

In [42]:
# d1=len(word_vocab)
# d2=len(tag_vocab)
# print(d1, d2) # 16881, 18

In [43]:
print(word_vocab.get_stoi())
print(tag_vocab.get_stoi())


{'\u200dराजधानी': 16880, '९७५१': 16875, '९३वीं': 16872, '९३,३६९': 16871, '९१४५': 16868, '९१': 16867, '९००': 16863, '९.५०': 16859, '९.४२': 16858, '९.४': 16857, '९.२': 16856, '९,५००': 16854, '८९१२.१२': 16851, '८९१': 16850, '८८८९': 16849, '८८५३.२१': 16848, '८२८७८१४': 16839, '८.७': 16834, '८.६७': 16833, '८.३५': 16830, '८.२५': 16829, '७९७५': 16827, '७९११': 16826, '७८वें': 16824, '७७८': 16823, '७७': 16821, '७५वीं': 16819, '७२.२८': 16816, '७१७.९': 16815, '७१६': 16813, '७१५': 16812, '७१३०': 16811, '७०००': 16808, '७०,१०१': 16807, '७.४३': 16804, '७.००': 16803, '६९१': 16800, '६८६': 16798, '६७८५': 16796, '६७५.८': 16795, '६७': 16794, '६६८.७५': 16793, '६५०': 16792, '६५.१२': 16791, '६४.४': 16789, '६२३': 16784, '६.८': 16776, '६.३०': 16773, '५९०२': 16767, '५९.४४': 16765, '५७.७': 16760, '५७.५९': 16759, '५४३': 16755, '५४०': 16754, '५३०': 16752, '५३.३२': 16751, '५२२३': 16750, '५२वीं': 16747, '५१७४': 16745, '५१६': 16744, '५०६४': 16743, '५.७': 16741, '४९६': 16735, '४९०': 16734, '४८९': 16732, '४६.५': 16725, 

In [44]:
"""word_vocab.add_item("कुश")"""

'word_vocab.add_item("कुश")'

In [45]:
#Normalize the data

def numericalize_data(data, vocab):
    #return [[vocab[word] for word in sentence] for sentence in data]
   return [[vocab[word] if word in vocab else vocab['<unk>'] for word in sentence] for sentence in data]

numericalized_train_words = numericalize_data(train_words, word_vocab)
numericalized_train_tags = numericalize_data(train_tags, tag_vocab)

numericalized_dev_words = numericalize_data(dev_words, word_vocab)
numericalized_dev_tags = numericalize_data(dev_tags, tag_vocab)

numericalized_test_words = numericalize_data(test_words, word_vocab)
numericalized_test_tags = numericalize_data(test_tags, tag_vocab)


In [46]:
print(len(numericalized_train_words),len(numericalized_train_tags))

13306 13306


In [111]:
print(numericalized_train_words)


[[26, 1341, 5, 193, 448, 14362, 4, 9, 22, 6, 3], [232, 12851, 15524, 8, 2456, 31, 3], [243, 926, 2584, 72, 5492, 6, 3], [339, 207, 14617, 16, 13, 264, 2444, 3406, 4, 61, 2, 19, 2896, 926, 2584, 16, 3], [188, 2091, 818, 793, 6, 3], [198, 1490, 529, 117, 130, 11, 10265, 376, 97, 2, 57, 7, 1567, 738, 6, 3], [1494, 882, 2, 256, 448, 1726, 2, 1510, 210, 9144, 11, 26, 793, 3594, 11013, 4095, 2, 2546, 11, 6, 1075, 11498, 4095, 18, 15, 200, 31, 3], [26, 1333, 13, 2469, 1710, 11, 9831, 9256, 6, 3], [26, 39, 5, 5787, 159, 1609, 6, 3], [264, 109, 9, 26, 616, 1710, 13, 3256, 1710, 11, 108, 6, 3], [232, 5137, 3201, 8, 2414, 23, 6, 3], [3084, 137, 4, 3640, 17, 904, 2, 1790, 25, 153, 11, 1078, 2673, 232, 13, 18, 5480, 13762, 6, 3], [2472, 12, 22, 1710, 1280, 14, 1710, 6293, 14, 8509, 10195, 2, 19, 7563, 14, 22, 12545, 14, 9762, 13, 13825, 10645, 14, 14910, 3887, 14, 95, 7536, 11, 8416, 14, 15522, 13, 2319, 3759, 1280, 18, 16, 3], [26, 257, 2, 157, 376, 583, 697, 7111, 281, 9, 260, 4881, 281, 41, 2256

In [47]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Pad sequences to avoid the mismatch
padded_train_words = pad_sequence([torch.tensor(seq) for seq in numericalized_train_words], padding_value=word_vocab['<pad>'])
padded_train_tags = pad_sequence([torch.tensor(seq) for seq in numericalized_train_tags], padding_value=tag_vocab['<pad>'])

padded_dev_words = pad_sequence([torch.tensor(seq) for seq in numericalized_dev_words], padding_value=word_vocab['<pad>'])
padded_dev_tags = pad_sequence([torch.tensor(seq) for seq in numericalized_dev_tags], padding_value=tag_vocab['<pad>'])

padded_test_words = pad_sequence([torch.tensor(seq) for seq in numericalized_test_words], padding_value=word_vocab['<pad>'])
padded_test_tags = pad_sequence([torch.tensor(seq) for seq in numericalized_test_tags], padding_value=tag_vocab['<pad>'])




In [84]:
print(len(padded_train_words), len(padded_train_tags))


116 116


In [106]:
padded_train_words[0]
print(padded_train_words.shape)
print(padded_train_words)

torch.Size([116, 13306])
tensor([[   26,   232,   243,  ...,    27,    77,    27],
        [ 1341, 12851,   926,  ...,    15,    79,   132],
        [    5, 15524,  2584,  ...,    10,   160,  4766],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])


In [48]:
#build a dataloader

from torch.utils.data import DataLoader, TensorDataset
def create_dataloader(inputs, targets, batch_size=32):
    dataset = TensorDataset(inputs, targets)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [49]:
'''train_inputs = padded_train_words.clone().detach()
train_targets = padded_train_tags.clone().detach()'''


'''dev_inputs = padded_dev_words.clone().detach()
dev_targets = padded_dev_tags.clone().detach()'''


'''test_inputs = padded_test_words.clone().detach()
test_targets = padded_test_tags.clone().detach()'''


'test_inputs = padded_test_words.clone().detach()\ntest_targets = padded_test_tags.clone().detach()'

In [50]:
train_inputs = torch.tensor(padded_train_words)
train_targets = torch.tensor(padded_train_tags)

#print("Targets tensor size:", train_targets.size())
dev_inputs = torch.tensor(padded_dev_words)
dev_targets = torch.tensor(padded_dev_tags)


test_inputs = torch.tensor(padded_test_words)
test_targets = torch.tensor(padded_test_tags)

<ipython-input-50-7a1318274098>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(padded_train_words)
<ipython-input-50-7a1318274098>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_targets = torch.tensor(padded_train_tags)
<ipython-input-50-7a1318274098>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dev_inputs = torch.tensor(padded_dev_words)
<ipython-input-50-7a1318274098>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().d

In [51]:
print(len(train_inputs),len(train_targets))
# print(train_targets.size(0))
# print(train_targets.size(1))

116 116


In [52]:
print(train_inputs.shape,train_targets.shape)
print(dev_inputs.shape,dev_targets.shape)
print(test_inputs.shape,test_targets.shape)

torch.Size([116, 13306]) torch.Size([116, 13306])
torch.Size([68, 1659]) torch.Size([68, 1659])
torch.Size([66, 1684]) torch.Size([66, 1684])


In [53]:

"""print(train_inputs.shape,train_targets.dim)
print(dev_inputs.shape,dev_targets.dim)
print(test_inputs.shape,test_targets.dim)"""

'print(train_inputs.shape,train_targets.dim)\nprint(dev_inputs.shape,dev_targets.dim)\nprint(test_inputs.shape,test_targets.dim)'

In [54]:

train_dataloader = create_dataloader(train_inputs, train_targets )
dev_dataloader = create_dataloader(dev_inputs, dev_targets)
test_dataloader =create_dataloader(test_inputs, test_targets)

In [55]:
print(len(train_dataloader), len(dev_dataloader), len(test_dataloader)) # batch sizes


4 3 3


In [56]:

print("Training DataLoader:")
for batch in train_dataloader:
    inputs, targets = batch
    print("Input Shape:", inputs.shape)
    print("Target Shape:", targets.shape)


Training DataLoader:
Input Shape: torch.Size([32, 13306])
Target Shape: torch.Size([32, 13306])
Input Shape: torch.Size([32, 13306])
Target Shape: torch.Size([32, 13306])
Input Shape: torch.Size([32, 13306])
Target Shape: torch.Size([32, 13306])
Input Shape: torch.Size([20, 13306])
Target Shape: torch.Size([20, 13306])


In [68]:


# Hyperparameters
embedding_dim =300
num_classes = len(tag_vocab )  # Number of POS tags
num_classes



18

In [69]:
import torch.nn as nn

class MultiClassLogisticRegression_PosTagger(nn.Module):
    def __init__(self, word_vocab, embedding_dim, num_classes):
        super(MultiClassLogisticRegression_PosTagger, self).__init__()
        self.embedding = nn.Embedding(word_vocab, embedding_dim)
        self.linear = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded_mean = torch.mean(embedded, dim=1)  # Average over the sequence dimension
        output = self.linear(embedded_mean)
        return output


In [70]:
model = MultiClassLogisticRegression_PosTagger(len(word_vocab), embedding_dim, num_classes)
model

MultiClassLogisticRegression_PosTagger(
  (embedding): Embedding(16881, 300)
  (linear): Linear(in_features=300, out_features=18, bias=True)
)

In [71]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [72]:
epochs = 5
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        output = model(inputs)
#print(output)
print(output.size())

torch.Size([20, 18])


In [73]:
print("Output shape:", output.shape)
print("Targets shape:", targets.shape)


Output shape: torch.Size([20, 18])
Targets shape: torch.Size([20, 13306])


In [74]:
#reshaped_target = targets.view(20, 18)


In [75]:
# Assuming one-hot encoded labels
# targets_indices = torch.argmax(targets, dim=1)
# targets_flattened = targets_indices.view(-1)
# loss = criterion(output, targets_flattened)


In [76]:
"""targets_flattened = targets.view(-1)
loss = criterion(output, targets_flattened)"""


'targets_flattened = targets.view(-1)\nloss = criterion(output, targets_flattened)'

In [81]:
#loss = criterion(output.view(-1, num_classes), targets.view(-1))
#loss = criterion(output, targets) #.view(-1)
loss = criterion(output, targets.view(-1))
loss.backward()
optimizer.step()


ValueError: Expected input batch_size (20) to match target batch_size (266120).

In [ ]:
# Validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_count = 0
        for inputs, targets in dev_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            output = model(inputs)
            _, predicted = torch.max(output, 1)
            total_correct += (predicted == targets.view(-1)).sum().item()
            total_count += targets.numel()

        accuracy = total_correct / total_count
        print(accuracy )

